In [10]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
# from mesa.batchrunner import BatchRunner
from model.model import City

In [11]:
# Create and run the model
city = City()

time_steps = 10
for t in range(time_steps):
    city.step()